In [4]:
// Add the serde and serde_json dependencies with the derive feature
:dep serde = { version = "1.0", features = ["derive"] }
:dep serde_json = "1.0"

// Use the crates
use serde::{Serialize, Deserialize};
use serde_json::json;

#[derive(Serialize, Deserialize)]
struct MyStruct {
    name: String,
    value: i32,
}

let my_data = MyStruct {
    name: String::from("example"),
    value: 42,
};

// Serialize to JSON with explicit type
let json_data: String = serde_json::to_string(&my_data).unwrap();
println!("{}", json_data);


{"name":"example","value":42}


In [11]:
// Add dependencies
:dep ndarray = "0.15" 
// No features
:dep rand = "0.8"

use ndarray::{Array, Array2, Axis};
use rand::Rng; // Import Rng trait

fn normalize(X: &Array2<f64>, epsilon: f64) -> (Array2<f64>, Array2<f64>, Array2<f64>) {
    let mean = X.mean_axis(Axis(1)).unwrap().insert_axis(Axis(1));
    let std = X.std_axis(Axis(1), 0.0).insert_axis(Axis(1)) + epsilon;

    let normalized = (X - &mean) / &std;
    (normalized, mean, std)
}

fn denormalize(A: &Array2<f64>, mean: &Array2<f64>, std: &Array2<f64>) -> Array2<f64> {
    A * std + mean
}

fn forward_propagation(X: &Array2<f64>, W: &Array2<f64>, b: f64) -> Array2<f64> {
    W.dot(X) + b
}

fn backward_propagation(X: &Array2<f64>, A: &Array2<f64>, Y: &Array2<f64>) -> (Array2<f64>, f64) {
    let m = X.shape()[1] as f64;
    let dZ = A - Y;
    let dW = dZ.dot(&X.t()) / m;
    let db = dZ.sum() / m;

    (dW, db)
}

fn compute_cost(Y: &Array2<f64>, A: &Array2<f64>) -> f64 {
    let m = Y.shape()[1] as f64;
    (1.0 / (2.0 * m)) * (A - Y).mapv(|x| x.powi(2)).sum()
}

fn neural_network(X: &Array2<f64>, Y: &Array2<f64>, learning_rate: f64, iterations: usize) -> (Array2<f64>, f64) {
    let n_x = X.shape()[0];
    let mut rng = rand::thread_rng();
    let mut W = Array::from_shape_vec((1, n_x), (0..n_x).map(|_| rng.gen_range(-0.01..0.01)).collect()).unwrap(); // Use rand for weights
    let mut b = 0.0;

    for i in 0..iterations {
        let A = forward_propagation(X, &W, b);
        let cost = compute_cost(Y, &A);

        if cost.is_nan() || cost.is_infinite() {
            println!("Stopping at iteration {} due to overflow", i);
            break;
        }

        let (dW, db) = backward_propagation(X, &A, Y);
        W = &W - &(learning_rate * dW);
        b -= learning_rate * db;

        if i % 100 == 0 {
            println!("Iteration {}, Cost: {}", i, cost);
        }
    }
    (W, b)
}

fn predict(X: &Array2<f64>, W: &Array2<f64>, b: f64) -> Array2<f64> {
    forward_propagation(X, W, b)
}

fn main() {
    // Example dataset: Rooms, Area (sqft), Floors, Age (years), Garage
    let X = Array::from_shape_vec((5, 6), vec![
        3.0, 4.0, 2.0, 5.0, 3.0, 6.0,  // Rooms
        1500.0, 1800.0, 800.0, 2300.0, 1200.0, 3000.0,  // Area
        2.0, 2.0, 1.0, 3.0, 1.0, 3.0,  // Floors
        10.0, 5.0, 20.0, 8.0, 30.0, 2.0,  // Age
        1.0, 1.0, 0.0, 1.0, 0.0, 1.0   // Garage (0 or 1)
    ]).unwrap();

    // Prices in $1000 (Target)
    let Y = Array::from_shape_vec((1, 6), vec![400.0, 500.0, 150.0, 650.0, 200.0, 900.0]).unwrap();

    // Normalize the input features
    let (X_normalized, X_mean, X_std) = normalize(&X, 1e-8);
    let (Y_normalized, Y_mean, Y_std) = normalize(&Y, 1e-8);

    // Train the neural network
    let (W, b) = neural_network(&X_normalized, &Y_normalized, 0.00001, 1000);

    // New input for prediction
    let p = Array::from_shape_vec((5, 1), vec![8.0, 18000.0, 2.0, 1.0, 2.0]).unwrap();
    
    // Normalize the new input for prediction
    let p_normalized = (&p - &X_mean) / &X_std;

    // Make predictions (normalized)
    let predicted_price_normalized = predict(&p_normalized, &W, b);

    // Denormalize the predicted price
    let predicted_price = denormalize(&predicted_price_normalized, &Y_mean, &Y_std);

    println!("Predicted Price: {}", predicted_price[[0, 0]]);
}


The type of the variable my_data was redefined, so was lost.


Iteration 0, Cost: 0.5080721728903961
Iteration 100, Cost: 0.5038466376677018
Iteration 200, Cost: 0.49965769210713795
Iteration 300, Cost: 0.49550501830411764
Iteration 400, Cost: 0.4913883011171367
Iteration 500, Cost: 0.48730722814375793
Iteration 600, Cost: 0.4832614896968035
Iteration 700, Cost: 0.4792507787807532
Iteration 800, Cost: 0.47527479106835013
Iteration 900, Cost: 0.4713332248774075
Predicted Price: 486.06099443078426


()